<h2 style="color: white;background-color: red;">Building user-based recommendation model for Amazon.</h2>

<h2 style="color: black;background-color: Lime;">Exploratory Data Analysis:</h2>

In [1]:
import pandas as pd
import numpy as np

# To deal with SettingWithCopyWarning in Pandas
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Load the Dataset
df = pd.read_csv('Amazon - Movies and TV Ratings.csv')
df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2 style="color: white;background-color: blue;">Which movies have maximum views/ratings?</h2>

In [3]:
df.describe()

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,Movie10,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
count,1.0,1.0,1.0,2.0,29.000000,1.0,1.0,1.0,1.0,1.0,...,5.000000,2.0,1.0,8.000000,3.000000,6.000000,1.0,8.000000,35.000000,13.000000
mean,5.0,5.0,2.0,5.0,4.103448,4.0,5.0,5.0,5.0,5.0,...,3.800000,5.0,5.0,4.625000,4.333333,4.333333,3.0,4.375000,4.628571,4.923077
std,NaN,NaN,NaN,0.0,1.496301,NaN,NaN,NaN,NaN,NaN,...,1.643168,0.0,NaN,0.517549,1.154701,1.632993,NaN,1.407886,0.910259,0.277350
min,5.0,5.0,2.0,5.0,1.000000,4.0,5.0,5.0,5.0,5.0,...,1.000000,5.0,5.0,4.000000,3.000000,1.000000,3.0,1.000000,1.000000,4.000000
25%,5.0,5.0,2.0,5.0,4.000000,4.0,5.0,5.0,5.0,5.0,...,4.000000,5.0,5.0,4.000000,4.000000,5.000000,3.0,4.750000,5.000000,5.000000
50%,5.0,5.0,2.0,5.0,5.000000,4.0,5.0,5.0,5.0,5.0,...,4.000000,5.0,5.0,5.000000,5.000000,5.000000,3.0,5.000000,5.000000,5.000000
75%,5.0,5.0,2.0,5.0,5.000000,4.0,5.0,5.0,5.0,5.0,...,5.000000,5.0,5.0,5.000000,5.000000,5.000000,3.0,5.000000,5.000000,5.000000
max,5.0,5.0,2.0,5.0,5.000000,4.0,5.0,5.0,5.0,5.0,...,5.000000,5.0,5.0,5.000000,5.000000,5.000000,3.0,5.000000,5.000000,5.000000


In [4]:
df.describe().T["count"].sort_values(ascending = False)[:10].to_frame()

,count
Movie127,2313.0
Movie140,578.0
Movie16,320.0
Movie103,272.0
Movie29,243.0
Movie91,128.0
Movie92,101.0
Movie89,83.0
Movie158,66.0
Movie108,54.0


<h2 style="color: black;background-color: yellow;">What is the average rating for each movie? </h2>

In [5]:
df.describe().T["mean"][:5].to_frame()

,mean
Movie1,5.000000
Movie2,5.000000
Movie3,2.000000
Movie4,5.000000
Movie5,4.103448


<h2 style="color: black;background-color: pink;">Define the top 5 movies with the maximum ratings.</h2>

In [6]:
avg_rating = df.describe().T["mean"]
avg_rating.head()

Movie1    5.000000
Movie2    5.000000
Movie3    2.000000
Movie4    5.000000
Movie5    4.103448
Name: mean, dtype: float64

In [7]:
rating_count = df.describe().T["count"]
rating_count.head()

Movie1     1.0
Movie2     1.0
Movie3     1.0
Movie4     2.0
Movie5    29.0
Name: count, dtype: float64

In [8]:
avg_rating_count = pd.concat([avg_rating, rating_count], axis=1, sort=False)
avg_rating_count.head()

,mean,count
Movie1,5.000000,1.0
Movie2,5.000000,1.0
Movie3,2.000000,1.0
Movie4,5.000000,2.0
Movie5,4.103448,29.0


In [9]:
C = avg_rating.mean()            #Mean vote average on the whole dataset
C

4.448436665448387

In [10]:
m = rating_count.quantile(0.85) # Minimum number of votes required to be listed
m

9.0

In [11]:
def wtdrt(x,m=m,C=C):
    V = x['count']
    R = x['mean']
    
    return (R*(V/(V+m)) + (C*(m/(V+m))))

In [12]:
df_movie_list_considered = avg_rating_count.loc[avg_rating_count['count']>=m]
df_movie_list_considered['Score'] = df_movie_list_considered.apply(wtdrt,axis=1)

In [13]:
df_movie_list_considered['Score'].sort_values(ascending=False).head(5)

Movie140    4.828000
Movie29     4.793793
Movie158    4.773812
Movie92     4.745781
Movie206    4.728906
Name: Score, dtype: float64

<h2 style="color: white;background-color: green;">Define the top 5 movies with the least audience.</h2>

In [14]:
top_movies_least_audience = avg_rating_count[avg_rating_count['count'] == 1]['mean'].head().sort_values(ascending=False)
top_movies_least_audience

Movie7    5.0
Movie2    5.0
Movie1    5.0
Movie6    4.0
Movie3    2.0
Name: mean, dtype: float64

<h2 style="color: black;background-color: Lime;">Recommendation Model:</h2>

In [15]:
df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Data Set Adjustments to use the recommendation Algo.

df_adjusted = df.melt(id_vars= df.columns[0], value_vars=df.columns[1:], var_name='MovieName',value_name='Rating')
df_adjusted.head()

,user_id,MovieName,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN


In [17]:
# Divide the Data into Training and Test data

In [18]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate, train_test_split

reader = Reader()
df_adjusted.fillna(df_adjusted['Rating'].mean(), inplace=True)

data = Dataset.load_from_df(df_adjusted, reader=reader)
trainData, testData = train_test_split(data, test_size=0.40)

In [19]:
# Recommendation Algo on Training Data.

algo = SVD()
algo.fit(trainData)

In [20]:
# Predictions on test data

accuracy.rmse(algo.test(testData))

RMSE: 0.0857


0.08574570553575005

In [21]:
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0798  0.0863  0.0886  0.0875  0.0867  0.0858  0.0031  
MAE (testset)     0.0090  0.0098  0.0099  0.0101  0.0097  0.0097  0.0004  
Fit time          44.72   45.04   44.92   44.26   45.05   44.80   0.29    
Test time         2.55    2.28    2.57    2.21    2.52    2.43    0.15    


{'test_rmse': array([0.07977236, 0.08626655, 0.08858559, 0.08752419, 0.08670604]),
 'test_mae': array([0.00903091, 0.00975352, 0.00990975, 0.0100576 , 0.00967337]),
 'fit_time': (44.72227168083191,
  45.03597021102905,
  44.92421364784241,
  44.26062870025635,
  45.05197811126709),
 'test_time': (2.547745704650879,
  2.2788686752319336,
  2.5690174102783203,
  2.2075133323669434,
  2.523447275161743)}

In [22]:
algo.predict('A3R5OBKS7OM2IR', 'Movie1', 5.0, verbose=True)

user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.39   {'was_impossible': False}


Prediction(uid='A3R5OBKS7OM2IR', iid='Movie1', r_ui=5.0, est=4.389878306942854, details={'was_impossible': False})

In [23]:
algo.predict('A3R5OBKS7OM2IR', 'Movie3', verbose=True)

user: A3R5OBKS7OM2IR item: Movie3     r_ui = None   est = 4.39   {'was_impossible': False}


Prediction(uid='A3R5OBKS7OM2IR', iid='Movie3', r_ui=None, est=4.38995449136945, details={'was_impossible': False})